In [ ]:
import numpy as np
import qiskit.pulse
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import qiskit.providers
import qiskit.quantum_info
from qiskit.quantum_info import Choi
import qiskit.quantum_info as qi


from qiskit import transpile, schedule
from qiskit.circuit import QuantumRegister, QuantumCircuit, Gate
from qiskit.pulse import Schedule, Play, ControlChannel
from qiskit.pulse import GaussianSquare
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.test.mock import FakeAlmaden


In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider (
    resource_id = "/subscriptions/b0c3bbe3-7123-4ba4-9aa2-57d9845f4a1d/resourceGroups/azurequantum/providers/Microsoft.Quantum/Workspaces/qchack-2022-b",
    location = "eastus"
)

In [ ]:
# Process tomography
q = QuantumRegister(3)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.x(q[1])
circ.cx(q[0],q[2])

# Get the ideal unitary operator
target_unitary = qi.Operator(circ)

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
quantinuum_sim_backend = provider.get_backend("quantinuum.hqs-lt-s2-sim")
job = qiskit.execute(qpt_circs, backend = backend, shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

/usr/local/lib/python3.7/site-packages/qiskit/test/mock/fake_backend.py:460: RuntimeWarning: Aer not found using BasicAer and no noise
  warnings.warn("Aer not found using BasicAer and no noise", RuntimeWarning)


{(('Zp', 'Zp', 'Zp'), ('X', 'X', 'X')): {'101': 1021,
  '000': 992,
  '010': 1038,
  '111': 949},
 (('Zp', 'Zp', 'Zp'), ('X', 'X', 'Y')): {'100': 545,
  '011': 496,
  '010': 516,
  '000': 467,
  '001': 521,
  '110': 476,
  '101': 531,
  '111': 448},
 (('Zp', 'Zp', 'Zp'), ('X', 'X', 'Z')): {'111': 503,
  '011': 476,
  '010': 500,
  '101': 540,
  '001': 513,
  '110': 480,
  '100': 502,
  '000': 486},
 (('Zp', 'Zp', 'Zp'), ('X', 'Y', 'X')): {'000': 1012,
  '010': 1020,
  '111': 978,
  '101': 990},
 (('Zp', 'Zp', 'Zp'), ('X', 'Y', 'Y')): {'001': 545,
  '010': 520,
  '110': 446,
  '000': 472,
  '011': 525,
  '101': 493,
  '111': 527,
  '100': 472},
 (('Zp', 'Zp', 'Zp'), ('X', 'Y', 'Z')): {'010': 488,
  '001': 518,
  '111': 490,
  '000': 488,
  '110': 525,
  '100': 500,
  '011': 483,
  '101': 508},
 (('Zp', 'Zp', 'Zp'), ('X', 'Z', 'X')): {'010': 2033, '111': 1967},
 (('Zp', 'Zp', 'Zp'), ('X', 'Z', 'Y')): {'111': 1044,
  '110': 983,
  '011': 967,
  '010': 1006},
 (('Zp', 'Zp', 'Zp'), ('X', 'Z

In [ ]:
import time
t = time.time()
choi_fit_lstsq = qpt_tomo.fit(method='lstsq')

print('Fit time:', time.time() - t)
print('Average gate fidelity: F = {:.5f}'.format(qi.average_gate_fidelity(choi_fit_lstsq, target=target_unitary)))

Fit time: 229.64727878570557
Average gate fidelity: F = 0.98737


In [ ]:
# Unique name for the instruction.
gate_name = 'cr1'

# Pulse parameters determined during calibration.
duration = 769
sigma = 42
square_width = 62
amp = 0.4
# Call the backend and get basis_gates and inst_map.
backend = FakeAlmaden()
config = backend.configuration()
defaults = backend.defaults()
basis_gates = config.basis_gates
inst_map = defaults.instruction_schedule_map

# Create the CR1 schedule.
cr1_pulse = GaussianSquare(duration, amp, sigma, square_width)
sched = Schedule()
sched += Play(cr1_pulse, ControlChannel(0))

# Add the CR1 instruction to basis_gates and inst_map.
basis_gates += [gate_name]
inst_map.add(gate_name, [1, 0], sched)

# Create a quantum gate to reference the CR1 pulse schedule.
cr1_gate = Gate(gate_name, 2, [])

# Create the QPT circuits.
qregs = QuantumRegister(3)
circuit = QuantumCircuit(qregs)
circuit.append(cr1_gate, qargs=[qregs[1], qregs[0]])
print (circuit)
qpt_circuits = process_tomography_circuits(circuit, [qregs[0], qregs[1]])

# Create the QPT pulse schedules.
qpt_circuits_1 = transpile(qpt_circuits, backend, basis_gates)
qpt_schedules = schedule(qpt_circuits_1, backend, inst_map)
#job = qiskit.execute(qpt_circuits, backend = backend, shots=5000)


       ┌──────┐
q11_0: ┤1     ├
       │  cr1 │
q11_1: ┤0     ├
       └──────┘
q11_2: ────────
               
